In [2]:
import numpy as np
import matplotlib.pyplot as plt
from src.utils import load_json

In [3]:
S = np.load("/home/spn1560/hiec/artifacts/sim_mats/sprhea_v3_folded_pt_ns_rcmcs_similarity_matrix.npy")
i2id = load_json("/home/spn1560/hiec/artifacts/sim_mats/sprhea_v3_folded_pt_ns_rcmcs_sim_i_to_id.json")
i2id = {int(k) : v for k, v in i2id.items()}
krs = load_json("/home/spn1560/hiec/data/sprhea/v3_folded_pt_ns.json")

In [4]:
# Histogram of sim scores
srt_s = sorted(S.ravel())
ycdf = np.arange(len(srt_s)) / len(srt_s)
plt.plot(srt_s, ycdf)

: 

: 

: 